In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import time
import matplotlib.pyplot as plt
from attrdict import AttrDict

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque
import random

In [ ]:
# Connect to PyBullet and set up the environment
p.connect(p.GUI)
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Optionally hide the PyBullet GUI

# Load the KUKA robot and environment objects
planeId = p.loadURDF("plane.urdf")
cuboid_green_id = p.loadURDF("./object/block.urdf", [0.54, 0, 0.02], [0, 0, 0, 1])
kuka_id = p.loadURDF("kuka_iiwa/kuka_with_prismatic_gripper.urdf")

p.setGravity(0, 0, -10)
p.resetDebugVisualizerCamera(
    cameraDistance=2.0,    # Zoom level (2 meters away)
    cameraYaw=75,          # Rotation around the vertical axis
    cameraPitch=-40,       # Tilt downward
    cameraTargetPosition=[0, 0, 0]  # Focus on the origin
)

numJoints = p.getNumJoints(kuka_id)  # Total joints: KUKA + gripper

# Initialize the joints dictionary
joints = AttrDict()

# Populate the joints dictionary with information about each joint
for joint_index in range(numJoints):
    joint_info = p.getJointInfo(kuka_id, joint_index)
    joint_name = joint_info[1].decode("utf-8")
    joints[joint_name] = AttrDict({
        "id": joint_info[0],
        "lowerLimit": joint_info[8],
        "upperLimit": joint_info[9],
        "maxForce": joint_info[10],
        "maxVelocity": joint_info[11],
    })

In [ ]:
class BulletKukaEnvironment(gym.Env):
    def __init__(self, gui=True):

        self.sim_time_step = 1 / 240  # TODO
        self.eff_index = 7
        self.home_kuka_joint_angle = np.array([0.0] * 7)
        self.init_kuka_joint_angle = np.array([-0., 0.44, 0., -2.086, -0., 0.615, -0.])

        self.action_space = spaces.Discrete(2)  # 0: xPlus, 1: xMinus
        self.observation_space = spaces.Box(low=np.array([0.0, -np.pi]), high=np.array([1.0, np.pi]), dtype=np.float32)

        self.step_count = 0
        self.max_steps = 20

    #     self.reset()

    def reset(self):

        self.set_kuka_joint_angles(self.home_kuka_joint_angle, self.init_kuka_joint_angle, duration=1)
        self.operate_gripper(init_pos=0., fin_pos=0.01, duration=1)  # Open gripper

        self.step_count = 0
        eef_pos, _ = self.get_current_eef_pose()
        obj_tilt = self.get_object_state(cuboid_green_id)[1]
        obs = np.array([eef_pos[0], obj_tilt], dtype=np.float32)
        return obs

    #     return self.get_observation()


    def calculate_ik(self, position, orientation):
        quaternion = p.getQuaternionFromEuler(orientation)
        lower_limits = [-np.pi] * 7
        upper_limits = [np.pi] * 7
        joint_ranges = [2 * np.pi] * 7
        rest_poses = [(-0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0)]  # Rest pose for 7 DOF
        # Calculate inverse kinematics with a 7-element damping vector
        joint_angles = p.calculateInverseKinematics(
            kuka_id, self.eff_index, position, quaternion,
            jointDamping=[0.01] * 7,
            lowerLimits=lower_limits,
            upperLimits=upper_limits,
            jointRanges=joint_ranges,
            restPoses=rest_poses
        )
        return joint_angles

    def set_kuka_joint_angles(self, init_joint_angles, des_joint_angles, duration):
        control_joints = ["lbr_iiwa_joint_1", "lbr_iiwa_joint_2", "lbr_iiwa_joint_3",
                        "lbr_iiwa_joint_4", "lbr_iiwa_joint_5", "lbr_iiwa_joint_6",
                        "lbr_iiwa_joint_7"]
        poses = []
        indexes = []
        forces = []
        for i, name in enumerate(control_joints):
            joint = joints[name]
            poses.append(des_joint_angles[i])
            indexes.append(joint.id)
            forces.append(joint.maxForce)
        trajectory = self.interpolate_trajectory(init_joint_angles, des_joint_angles, duration)
        for q_t in trajectory:
            p.setJointMotorControlArray(
                kuka_id, indexes,
                controlMode=p.POSITION_CONTROL,
                targetPositions=q_t,
                forces=forces
            )
            p.stepSimulation()
            time.sleep(self.sim_time_step)

    def position_path(self, t, t_max, start_pos, end_pos):
        return start_pos + (end_pos - start_pos) * (t / t_max)

    def orientation_path(self, t, t_max, start_orient, end_orient):
        """Orientation path (Euler angles)."""
        return start_orient + (end_orient - start_orient) * (t / t_max)

    def get_current_eef_pose(self):
        linkstate = p.getLinkState(kuka_id, self.eff_index, computeForwardKinematics=True)
        position, orientation = linkstate[0], linkstate[1]
        position = list(position)
        position[2] = position[2] - 0.0491
        return (position, list(p.getEulerFromQuaternion(orientation)))

    def get_current_joint_angles(self, kuka_or_gripper=None):
        joint_states = p.getJointStates(kuka_id, list(range(numJoints)))
        joint_values = [state[0] for state in joint_states]
        if kuka_or_gripper == 'kuka':
            return joint_values[:7]  # First 7 joints for the KUKA arm
        elif kuka_or_gripper == 'gripper':
            return joint_values[7:]  # Remaining joints for the gripper
        else:
            return joint_values

    def interpolate_trajectory(self, q_start, q_end, duration):
        num_steps = int(duration / self.sim_time_step) + 1
        trajectory = []
        for t in range(num_steps):
            alpha = t / (num_steps - 1)
            q_t = [q_start[i] + alpha * (q_end[i] - q_start[i]) for i in range(len(q_start))]
            trajectory.append(q_t)
        return trajectory

    def moveL(self, start_pos, end_pos, target_ori, duration):
        """
        Moves the robot's end-effector in a straight line from start_pos to end_pos.
        """
        num_steps = int(duration / self.sim_time_step) + 1
        target_quat = p.getQuaternionFromEuler(target_ori)
        for step in range(num_steps):
            alpha = step / (num_steps - 1)
            current_pos = np.array(start_pos) + alpha * (np.array(end_pos) - np.array(start_pos))
            # Calculate inverse kinematics for the current target position and fixed orientation
            joint_poses = p.calculateInverseKinematics(kuka_id, self.eff_index, current_pos.tolist(), target_quat)
            # Assuming that the arm joints are the first 7 joints
            control_joint_indices = list(range(7))
            target_positions = joint_poses[:7]
            p.setJointMotorControlArray(kuka_id, control_joint_indices,
                                        controlMode=p.POSITION_CONTROL,
                                        targetPositions=target_positions)
            p.stepSimulation()
            time.sleep(self.sim_time_step)

    def execute_task_space_trajectory(self, start_pos, final_pos, duration=1):
        # final_pos is a two-element structure: [position, orientation]
        all_joint_angles = self.calculate_ik(final_pos[0], final_pos[1])
        des_kuka_joint_angles = all_joint_angles[:7]  # Use the first 7 joint angles
        self.set_kuka_joint_angles(self.get_current_joint_angles("kuka"), des_kuka_joint_angles, duration)

    def operate_gripper(self, init_pos, fin_pos, duration=1):
        """
        Smoothly open or close the gripper by interpolating the gripper opening angle.
        Args:
            duration (float): Duration for the gripper motion (seconds).
            init_pos (float): Initial gripper opening distance.
            fin_pos (float): Final gripper opening distance.
        """
        control_joints = ["left_finger_sliding_joint", "right_finger_sliding_joint"]
        poses = []
        indexes = []
        forces = []
        
        init_arr = np.array([-init_pos, init_pos])
        fin_arr = np.array([-fin_pos, fin_pos])
        for i, name in enumerate(control_joints):
            joint = joints[name]
            poses.append(fin_arr[i])
            indexes.append(joint.id)
            forces.append(joint.maxForce)
        trajectory = self.interpolate_trajectory(init_arr, fin_arr, duration)
        for q_t in trajectory:
            p.setJointMotorControlArray(
                kuka_id, indexes,
                controlMode=p.POSITION_CONTROL,
                targetPositions=q_t,
                forces=forces
            )
            p.stepSimulation()
            time.sleep(self.sim_time_step)

    def get_object_state(self, object_id):
        position, orientation = p.getBasePositionAndOrientation(object_id)
        orientation_euler = p.getEulerFromQuaternion(orientation)
        return orientation_euler 
        

    def execute_grip_action(self):
        """ 
        Move the gripper down, grip the object, lift it up and put it down again, 
        finally bring the gripper up to its initial position.
        """

        current_eef_pos, current_eef_orien = self.get_current_eef_pose()
        next_eef_pos = current_eef_pos.copy()
        next_eef_pos[2] = next_eef_pos[2] - 0.1225

        self.moveL(current_eef_pos, next_eef_pos, current_eef_orien, duration=1)
        self.operate_gripper(init_pos=0.01, fin_pos=0.0008, duration=0.5)  # Close gripper
        self.moveL(next_eef_pos, current_eef_pos, current_eef_orien, duration=1)
        obj_tilt_angle = self.get_object_state(cuboid_green_id)[1]
        self.moveL(current_eef_pos, next_eef_pos, current_eef_orien, duration=1)
        self.operate_gripper(init_pos=0., fin_pos=0.01, duration=1)  # Open gripper
        self.moveL(next_eef_pos, current_eef_pos, current_eef_orien, duration=1)

        return obj_tilt_angle

    # def execute_grip_action(self):
    #     """ 
    #     Move the gripper down, grip the object, lift it up and put it down again, 
    #     finally bring the gripper up to its initial position. Also detects if the object was grasped.
    #     """
    #     current_eef_pos, current_eef_orien = self.get_current_eef_pose()
    #     grasp_pos = current_eef_pos.copy()
    #     grasp_pos[2] -= 0.1225  

    #     self.moveL(current_eef_pos, grasp_pos, current_eef_orien, duration=1)
    #     self.operate_gripper(init_pos=0.01, fin_pos=0.0008, duration=0.5)

    #     before_lift_pos, _ = p.getBasePositionAndOrientation(cuboid_green_id)
    #     self.moveL(grasp_pos, current_eef_pos, current_eef_orien, duration=1)

    #     after_lift_pos, _ = p.getBasePositionAndOrientation(cuboid_green_id)

    #     lifted = (after_lift_pos[2] - before_lift_pos[2]) > 0.015

    #     obj_tilt_angle = self.get_object_state(cuboid_green_id)[1]

    #     self.moveL(current_eef_pos, grasp_pos, current_eef_orien, duration=1)
    #     self.operate_gripper(init_pos=0., fin_pos=0.01, duration=1)  # Open gripper
    #     self.moveL(grasp_pos, current_eef_pos, current_eef_orien, duration=1)

    #     return obj_tilt_angle, lifted


    

    def step(self, action):
        self.step_count += 1

        x_step = 0.05
        current_eef_pos, current_eef_orien = self.get_current_eef_pose()
        next_eef_pos = current_eef_pos.copy()

        if action == 0:  # xPlus
            next_eef_pos[0] += x_step
        elif action == 1:  # xMinus
            next_eef_pos[0] -= x_step

        self.moveL(current_eef_pos, next_eef_pos, current_eef_orien, duration=2)

        # tilt, lifted = self.execute_grip_action()
        tilt = self.execute_grip_action()

        obj_pos, _ = p.getBasePositionAndOrientation(cuboid_green_id)
        obs = np.array([next_eef_pos[0], tilt], dtype=np.float32)

        tilt_penalty = abs(tilt) * 5
        height_reward = max(0, obj_pos[2] - 0.03) * 10  
        position_reward = -abs(next_eef_pos[0] - 0.54) * 2 
        # grasp_bonus = 5.0 if lifted else 0.0

        # reward = height_reward - tilt_penalty + position_reward + grasp_bonus
        reward = height_reward - tilt_penalty + position_reward

        terminated = (self.step_count >= self.max_steps)

        # Debug info
        info = {
            "tilt_angle": tilt,
            "obj_height": obj_pos[2],
            # "lifted": lifted
        }

        return obs, reward, terminated, info


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6, beta_start=0.4, beta_frames=1000):
        self.capacity = capacity
        self.alpha = alpha 
        self.beta_start = beta_start
        self.beta_frames = beta_frames
        self.frame = 1 
        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0
    
    def beta_by_frame(self, frame_idx):
 
        return min(1.0, self.beta_start + frame_idx * (1.0 - self.beta_start) / self.beta_frames)
    
    def push(self, state, action, reward, next_state, done):

        max_priority = self.priorities.max() if self.buffer else 1.0
        
        experience = (state, action, reward, next_state, done)
        
        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.position] = experience
        
        self.priorities[self.position] = max_priority
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        if len(self.buffer) == self.capacity:
            priorities = self.priorities
        else:
            priorities = self.priorities[:len(self.buffer)]
        
        priorities = np.maximum(priorities, 1e-5)
        
        probabilities = priorities ** self.alpha
        probabilities /= probabilities.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probabilities)
        
        beta = self.beta_by_frame(self.frame)
        self.frame += 1
        
        weights = (len(self.buffer) * probabilities[indices]) ** (-beta)
        weights /= weights.max() 
        weights = np.array(weights, dtype=np.float32)
        
        samples = [self.buffer[idx] for idx in indices]
        
        states, actions, rewards, next_states, dones = zip(*samples)
        
        states = torch.FloatTensor(np.array(states))
        next_states = torch.FloatTensor(np.array(next_states))
        actions = torch.LongTensor(np.array(actions)).unsqueeze(1)
        rewards = torch.FloatTensor(np.array(rewards)).unsqueeze(1)
        dones = torch.FloatTensor(np.array(dones)).unsqueeze(1)
        weights = torch.FloatTensor(weights).unsqueeze(1)
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority
    
    def __len__(self):
        return len(self.buffer)


class PERDQN:
    def __init__(self, env, gamma=0.99, batch_size=16):
        self.env = env
        self.state_dim = 2  
        self.action_dim = 2  
        
        self.gamma = gamma
        self.batch_size = batch_size
        self.target_update_freq = 10  
        self.learning_rate = 1e-3 
        
        self.Q_policy = NeuralNetwork(self.state_dim, self.action_dim)
        self.Q_prime = NeuralNetwork(self.state_dim, self.action_dim)
        self.Q_prime.load_state_dict(self.Q_policy.state_dict())
        self.Q_prime.eval()
        
        self.optimizer = torch.optim.Adam(self.Q_policy.parameters(), lr=self.learning_rate)
        self.replay_buffer = PrioritizedReplayBuffer(capacity=2000) 
        
        self.total_steps = 0
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.9  
        self.rewards_history = []

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)
        
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        else:
            with torch.no_grad():
                q_values = self.Q_policy(state)
                return q_values.max(1)[1].item()

    def update_model(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        states, actions, rewards, next_states, dones, indices, weights = self.replay_buffer.sample(self.batch_size)

        q_values = self.Q_policy(states).gather(1, actions)

        # next_actions = self.Q_policy(next_states).max(1)[1].unsqueeze(1)
        # next_q_values = self.Q_prime(next_states).gather(1, next_actions)

        next_q_values = self.Q_prime(next_states).max(1)[0].unsqueeze(1)

        expected_q_values = rewards + self.gamma * next_q_values * (1 - dones)

        td_errors = torch.abs(q_values - expected_q_values).detach().cpu().numpy()
        loss = (weights * F.smooth_l1_loss(q_values, expected_q_values, reduction='none')).mean()

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.Q_policy.parameters(), 1.0)
        self.optimizer.step()

        self.replay_buffer.update_priorities(indices, td_errors + 1e-5)

        return loss.item()

    def train(self, num_episodes=500):
        for episode in range(num_episodes):
            state = self.env.reset()
            episode_reward = 0
            actions_taken = []

            for step in range(self.env.max_steps):
                action = self.select_action(state)
                actions_taken.append(action)
                next_state, reward, done, info = self.env.step(action)
                episode_reward += reward

                self.replay_buffer.push(state, action, reward, next_state, done)

                if len(self.replay_buffer) >= self.batch_size:
                    self.update_model()

                if self.total_steps % self.target_update_freq == 0:
                    self.Q_prime.load_state_dict(self.Q_policy.state_dict())

                state = next_state
                self.total_steps += 1

                if done:
                    break

            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
            self.rewards_history.append(episode_reward)

            avg_reward = np.mean(self.rewards_history[-10:]) if len(self.rewards_history) >= 10 else np.mean(self.rewards_history)
            print(f"Episode {episode+1}/{num_episodes} | Reward: {episode_reward:.2f} | Avg Reward: {avg_reward:.2f} | Epsilon: {self.epsilon:.4f}")
            print(f"Actions taken: {actions_taken}")

        print("Training complete!")
        return self.rewards_history

    

In [ ]:
env = BulletKukaEnvironment()
agent = PERDQN(env)
rewards = agent.train(num_episodes=100)